In [14]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from difflib import SequenceMatcher as SM
import pandas as pd
import numpy as np
import time
import Levenshtein
import re
from newsapi import NewsApiClient
import requests
import bs4 as bs
from bs4.element import Comment

In [5]:
fuzz.ratio("this is a test", "this is a test!")
fuzz.token_sort_ratio("Amazon", "aMZon")

!pip install newsapi-python

You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
data = pd.read_csv('dataXData.csv')
company = data['Business Name'].unique()
company = company.astype(str)
data.head()

,Unnamed: 0,Business Name,Industry,City,State
0,0,3M Company,Industrials,NaN,USA
1,1,A.O. Smith Corp,Industrials,NaN,USA
2,2,Abbott Laboratories,Health Care,NaN,USA
3,3,AbbVie Inc.,Health Care,NaN,USA
4,4,Accenture plc,Information Technology,NaN,USA


In [7]:
pd.unique(data[['City']].values.ravel())

array([nan, 'Quebec', 'BC', ..., 'TEMPERANCE', 'DELMONT', 'MONACA'],
      dtype=object)

In [8]:
print(len(company))

940427


In [9]:
start=time.time()
response='y'
while(response=='y'):
    a=[]
    text2=input("Which text do you want to search? ")
    text1 = re.sub('[^A-Za-z0-9]+', '', text2)
    company = data['Business Name'].unique()
    company = company.astype(str)
    for i in range(0,len(company),1):
        search=company[i].lower()
        a.append(fuzz.token_sort_ratio(text1,search))

    #Finding the company names with maximum risk score aka fuzzy ratio
    ratio=np.array(a)
    c=np.where(ratio==ratio.max())
    maximum=max(a)
    top_list = company[c]
    print("Risk Score for",text2,"is",maximum,"for the brands",top_list)
    
    #Decision Logic for Classification
    if maximum>=90:
        print("High risk")
    elif (maximum<90 and maximum>=60):
        print("Medium Risk")
    else:
        print("No Risk")

    #Recording time taken to search the database
    end=time.time()
    print("Time taken:",round(end-start,3),"seconds")

    df=data.loc[data['Business Name'].isin(top_list)][['Business Name','Industry','City','State']]
    df1=df.drop_duplicates(keep='first')
    print(df1)
    response=input("Do you want to search again? [y/n] ")
print("Thanks for searching with us!")  

Which text do you want to search? apple inc
Risk Score for apple inc is 94 for the brands ['Apple Inc.' 'APPLE INC.' 'APPLE INC' 'Apple Inc' 'APPLE, INC.'
 'APPLE  INC']
High risk
Time taken: 33.582 seconds
        Business Name                                           Industry  \
51         Apple Inc.                             Information Technology   
2809       Apple Inc.                             Computer Manufacturing   
24144      APPLE INC.                                       RETAIL STORE   
43145       APPLE INC                    Other Computer Related Services   
114846      Apple Inc                                   Computer Systems   
232237      Apple Inc                                       Retail Trade   
352361    APPLE, INC.                         Computer & software stores   
644127    APPLE, INC.  All other miscellaneous store retailers (inclu...   
785676     APPLE  INC               All Other General Merchandise Stores   
798189    APPLE, INC.            

In [10]:
unique_names = []
for d in top_list:
    lower = d.lower()
    if lower not in unique_names:
        unique_names.append(lower)
        
print(unique_names)

['apple inc.', 'apple inc', 'apple, inc.', 'apple  inc']


In [20]:
def text_from_html(body):
    soup = bs.BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    # encode to get rid of encoding errors (hopefully)
    return " ".join(t.strip() for t in visible_texts).encode('utf-8','ignore')

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

In [38]:
from newsapi import NewsApiClient

entity_column = []
url_column = []
snippet_column = []
# Init
newsapi = NewsApiClient(api_key='a28acbbe0b2a46428906f353d180c5c3')
all_articles = []
for entity in unique_names:
    all_articles=newsapi.get_everything(q=entity,
                                         language='en', # necessary to only maintain english? probably for now
                                         sort_by='relevancy',
                                         page=1)["articles"]

    for article in all_articles:
        url = article["url"]        
        source = requests.get(url)
        if source.status_code != 200:
            continue
        text = text_from_html(source.content).decode('utf-8')
        
        if (' ' not in entity):
            splitted = text.split()
            lower = [x.lower() for x in splitted]

            try:
                index = lower.index(entity);

            except ValueError:
                index = -1
            if(index != -1):
                if(index < 50):
                    article_snippet = " ".join(splitted[0:index+50])
                else:
                    article_snippet = " ".join(splitted[index-50:index+50])
                entity_column.append(entity)
                url_column.append(url)
                snippet_column.append(article_snippet)
        else:
            indices = [m.start() for m in re.finditer(entity.lower(), text.lower())]
            
            if not indices: 
                continue
            index = indices[0]
            if(index < 100):
                article_snippet = text[0:index+100]
            else:
                article_snippet = text[index-100:index+100]
            entity_column.append(entity)
            url_column.append(url)
            snippet_column.append(article_snippet)
            
                
columns = {"entity" : entity_column, "url": url_column, "snippet" : snippet_column}
df_result = pd.DataFrame.from_dict(columns)

        
        


In [39]:
pd.set_option('max_colwidth', 300)

In [41]:
from IPython.display import display

display(df_result)

,entity,url,snippet
0,apple inc.,https://www.reuters.com/article/us-apple-products-idUSKCN1NF00Z,"pple Store in Manhattan, New York, U.S., September 21, 2018. REUTERS/Shannon Stapleton (Reuters) - Apple Inc said on Friday it had found some issues affecting some of its iPhone X and 13-inch MacBook"
1,apple inc.,https://www.fastcompany.com/90272180/supreme-court-hears-arguments-over-apple-app-store-monopoly?partner=feedburner&utm_source=feedburner&utm_medium=feed&utm_campaign=Feed%3A+fastcompany%2Fheadlines+%28Fast+Company%29,"e Court today to defend its 30 percent commission on app sales. The justices will hear arguments in Apple Inc. v. Pepper , over whether the tech giant can be forced to pay damages to iPhone owners who"
2,apple inc.,https://www.reuters.com/article/us-global-markets/asian-shares-rise-as-trump-xi-lift-hopes-on-resolving-trade-row-idUSKCN1N7033,orld’s two biggest economies were mending their shaky trade relations. A steep decline in shares of Apple Inc ( AAPL.O ) further weighed on sentiment in the U.S. stock market after the iPhone maker wa
3,apple inc.,https://www.reuters.com/article/us-apple-iphone-pricing-analysis-idUSKCN1NJ26G,"hangs over Apple Stephen Nellis 8 Min Read SAN FRANCISCO (Reuters) - A raft of profit warnings from Apple Inc ( AAPL.O ) suppliers this week has fueled investor concerns that iPhone sales, in terms of"
4,apple inc.,https://gizmodo.com/should-the-supreme-court-knock-the-first-brick-out-of-a-1830569176,nock the First Brick Out of Apple's Walled Garden? Rhett Jones 11/23/18 10:00am Filed to: Pepper v. Apple Inc Filed to: Pepper v. Apple Inc Pepper v. Apple Inc supreme court apple 157 4 Edit Embiggen
5,apple inc.,https://gizmodo.com/supreme-court-appears-to-lean-heavily-against-apples-de-1830662533,esent at today’s hearing say the case’s outcome isn’t looking great for Apple. The case of Pepper v Apple Inc has taken a complex path to the highest court in the land (which we’ve previously covered
6,apple inc.,https://www.entrepreneur.com/article/322750,) -- was down on the day. The Entrepreneur Index™ as a whole declined 0.23 percent. Tech bellwether Apple Inc. failed to add fuel to the post-October rally with financial results it reported last nigh
7,apple inc.,https://www.androidcentral.com/samsung-expected-open-bixby-developers-alexa-skills,"for help in bolstering its challenge to the industry's big three players: Amazon.com Inc.'s Alexa, Apple Inc.'s Siri and Alphabet Inc.'s Google Assistant. Furthermore, Samsung will announce ""capsul"
8,apple inc.,https://www.reuters.com/article/us-usa-stocks/wall-street-snaps-three-day-rally-as-apple-falls-trade-optimism-fades-idUSKCN1N7190,"llowing a disappointing forecast and the White House dampened optimism over U.S.-China trade talks. Apple Inc ( AAPL.O ) tumbled 6.6 percent, sending its market value below $1 trillion at the close, a"
9,apple inc.,https://www.reuters.com/article/us-usa-stocks/wall-street-snaps-three-day-rally-as-apple-falls-trade-hopes-fade-idUSKCN1N7190,"llowing a disappointing forecast and the White House dampened optimism over U.S.-China trade talks. Apple Inc ( AAPL.O ) tumbled 6.6 percent, sending its market value below $1 trillion at the close, a"


In [42]:
print(df_result.to_string())

        entity                                                                                                                                                                                                                        url                                                                                                                                                                                                   snippet
0   apple inc.                                                                                                                                                            https://www.reuters.com/article/us-apple-products-idUSKCN1NF00Z  pple Store in Manhattan, New York, U.S., September 21, 2018.  REUTERS/Shannon Stapleton (Reuters) - Apple Inc said on Friday it had found some issues affecting some of its iPhone X and 13-inch MacBook
1   apple inc.  https://www.fastcompany.com/90272180/supreme-court-hears-arguments-over-apple-app-store-monopoly?partner=feedbur